In [1]:
"""
Author 1: Sai Mahesh
Roll No: CS18BTECH11001

Author 2: Vamshi Krishna
Roll No: CS18BTECH11024

Author 3: Sai Varshittha
Roll No: CS18BTECH11035
"""

'\nAuthor 1: Sai Mahesh\nRoll No: CS18BTECH11001\n\nAuthor 2: Vamshi Krishna\nRoll No: CS18BTECH11024\n\nAuthor 3: Sai Varshittha\nRoll No: CS18BTECH11035\n'

In [2]:
# Modules to be used
import copy
import numpy as np
import pandas as pd
import pathlib
from tqdm import tqdm
from collections import defaultdict
from functools import reduce

In [3]:
# Dataset to be used for importing stock flow graph
PATH1 = pathlib.Path('./cluster.csv')
PATH2 = pathlib.Path('./example.csv')

In [4]:
# function to extract graph from dataset
def get_graph_from_dataset(path: pathlib.Path):
    # read data from given file
    df = pd.read_csv(path)
    df.columns = ['vertex1','vertex2','cost']
    
    # extract vertices
    # vertices = np.union1d(df['vertex1'],df['vertex2'])
    vertices = np.unique(df['vertex1'].to_numpy())
    graph = defaultdict(list)

    # fill graph
    for index, row in df.iterrows():
        vertex1, vertex2, cost = row
        graph[vertex1].append((vertex2, cost))

    return vertices, graph

In [5]:
# Base class for Nearest Neighbour
class NearestNeighbour:
    # Constructor
    def __init__(self, k, vertices, graph):
        self.k = k
        self.vertices = vertices
        self.graph = copy.copy(graph)
        self.kNN = None 

    # function for finding kNN 
    def getkNN(self):
        self.kNN = self.graph
        for v in self.vertices:
            self.kNN[v].sort(key=lambda x: x[1], reverse=True)
            self.kNN[v] = self.kNN[v][:self.k]
        return self.kNN

In [6]:
# function to merge all cluster tuples to single set
def merge_tuples_to_set(cluster):
    return set(reduce(lambda a,b: set(a).union(set(b)),cluster))

# Shared Nearest Neighbour

In [7]:
# Shared Nearest Neighbour
class SharedNearestNeighbour(NearestNeighbour):
    # Constructor
    def __init__(self, k, vertices, graph, kt):
        NearestNeighbour.__init__(self, k, vertices, graph)
        self.kt = kt
    
    # function for finding collusion set using shared_NN
    def shared_NN(self):
        # get kNN and convert into set datastructure for ease
        kNN_graph = self.getkNN()
        for v in self.vertices:
            kNN_graph[v] = set(map(lambda x: x[0],kNN_graph[v]))

        # S contains Collusion Sets
        # initially every vertex is a singleton cluster
        S = set(map(lambda x: tuple([x]),self.vertices))

        # Cluster of vertices
        # initially every vertex belongs to same singleton cluster
        C = defaultdict(set)
        for v in self.vertices:
            C[v] |= {v}
        
        # update collusion set as requried
        for u in self.vertices:
            for v in self.vertices:
                s1 = kNN_graph[u]
                s2 = kNN_graph[v]
                if (v not in C[u]) and (u in s2) and (v in s1) and (len(s1.intersection(s2))>=self.kt):
                    S.remove(tuple(C[u]))
                    S.remove(tuple(C[v]))
                    cluster_union = C[u].union(C[v])
                    S.add(tuple(cluster_union))
                    for vertex in cluster_union:
                        C[vertex] = cluster_union

        # Consider non-singleton collusion sets
        non_singleton_collusion_set = list(filter(lambda x: len(x)>1, S))
        sorted(non_singleton_collusion_set,key=lambda x: len(x),reverse=True)
        return non_singleton_collusion_set

In [8]:
# Function for finding candidate collusion set
def candidate_collusion_set(vertices, graph, hyper_parameters):
    frauds = []
    for k, kt in tqdm(hyper_parameters):
        print("\n\nRunning on Hyper Parameters: k = ",k," \t kt = ",kt)
        shared_obj = SharedNearestNeighbour(k,vertices,graph,kt)
        collusions = shared_obj.shared_NN()
        print("\nCollusion Sets:",collusions)
        collusionset = merge_tuples_to_set(collusions)
        frauds.append(collusionset)
        del shared_obj
    
    # return the intersection of vertices
    return set(reduce(lambda a,b: a.intersection(b), frauds))

In [9]:
# Testing the functions
hyper_parameters = [(4,2),(5,2),(5,3),(6,4),(7,4)]
vertices, graph = get_graph_from_dataset(PATH2)
print("\n\nCandidate Collusion Set: ",candidate_collusion_set(vertices,graph,hyper_parameters))

100%|██████████| 5/5 [00:00<00:00, 1811.64it/s]



Running on Hyper Parameters: k =  4  	 kt =  2

Collusion Sets: [(1042, 1074, 1029, 1035, 1037)]


Running on Hyper Parameters: k =  5  	 kt =  2

Collusion Sets: [(1042, 1074, 1029, 1035, 1037)]


Running on Hyper Parameters: k =  5  	 kt =  3

Collusion Sets: [(1042, 1074, 1029, 1035, 1037)]


Running on Hyper Parameters: k =  6  	 kt =  4

Collusion Sets: [(1037, 1042, 1074, 1029)]


Running on Hyper Parameters: k =  7  	 kt =  4

Collusion Sets: [(1057, 1042, 1074, 1029, 1049, 1037)]


Candidate Collusion Set:  {1042, 1029, 1074, 1037}


# Mutual Nearest Neighbour

In [10]:
# Mutual Nearest Neighbour
class MutualNearestNeighbour(NearestNeighbour):
    # Constructor
    def __init__(self, k, vertices, graph, m):
        NearestNeighbour.__init__(self, k, vertices, graph)
        self.m = m
        self.MAX_MNV = self.k*10
        self.mnvSet = defaultdict(dict)
        self.mnvClusterDP = defaultdict(dict)
        self.mnvPointsDP = defaultdict(dict)

    # function to find mnv and intercluster distance of two points
    def mnv_points(self, vertex1, vertex2):
        # Check if it is previously calculated
        if vertex1 in self.mnvPointsDP and vertex2 in self.mnvPointsDP[vertex1]:
            return_val = self.mnvPointsDP[vertex1][vertex2]
            return return_val[0], return_val[1]

        mnv, point_distance = 0, 1
        # If vertex1 is in kNN of vertex2
        if vertex1 in self.mnvSet[vertex2]:
            point_distance = min(point_distance, - self.mnvSet[vertex2][vertex1][1])
            mnv += self.mnvSet[vertex2][vertex1][0]
        else:
            mnv += self.MAX_MNV
        
        # If vertex2 is in kNN of vertex1
        if vertex2 in self.mnvSet[vertex1]:
            point_distance = min(point_distance, - self.mnvSet[vertex1][vertex2][1])
            mnv += self.mnvSet[vertex1][vertex2][0]
        else:
            mnv += self.MAX_MNV
        
        # Store the new values
        self.mnvPointsDP[vertex1][vertex2] = (mnv, point_distance)
        self.mnvPointsDP[vertex2][vertex1] = (mnv, point_distance)
        return mnv, point_distance
    
    # function to find mnv and intercluster distance of two clusters
    def mnv_clusters(self, cluster1, cluster2):
        # Check if it is previously calculated
        if cluster1 in self.mnvClusterDP and cluster2 in self.mnvClusterDP[cluster1]:
            return_val = self.mnvPoints[cluster1][cluster2]
            return return_val[0], return_val[1]

        mnv, inter_cluster_distance = 0, 1
        for vertex1 in cluster1:
            for vertex2 in cluster2:
                mnv_point, point_distance = self.mnv_points(vertex1, vertex2)
                mnv += mnv_point
                inter_cluster_distance = min(inter_cluster_distance, point_distance)
        mnv /= (len(cluster1)*len(cluster2))

        # Store the new values
        self.mnvPointsDP[cluster1][cluster2] = (mnv, inter_cluster_distance)
        self.mnvPointsDP[cluster2][cluster1] = (mnv, inter_cluster_distance)
        return mnv, inter_cluster_distance

    # function for finding collusion set using mutual_NN_avg
    def mutual_NN_avg(self):
        # get kNN and convert into set datastructure for ease
        # Set rank and amount for each edge in the kNN
        kNN_graph = self.getkNN()
        for v in self.vertices:
            for index, element in enumerate(kNN_graph[v]):
                self.mnvSet[v][element[0]] = (index + 1, element[1])
            kNN_graph[v] = set(map(lambda x: x[0],kNN_graph[v]))

        # S contains Collusion Sets
        # initially every vertex is a singleton cluster
        S = set(map(lambda x: tuple([x]),self.vertices))

        # update collusion set as required
        while len(S) > self.m:
            # print("len(S): ",len(S))
            min_mnv = 2*self.MAX_MNV
            min_inter_cluster_distance = 1
            min_cluster_pair = None
            list_S = list(S)
            for i in range(len(list_S)):
                for j in range(i+1,len(list_S)):
                    cluster1 = list_S[i]
                    cluster2 = list_S[j]
                    if not cluster1 == cluster2:
                        mnv, inter_cluster_distance = self.mnv_clusters(cluster1, cluster2)
                        if mnv < min_mnv or (mnv == min_mnv and inter_cluster_distance < min_inter_cluster_distance):
                            min_inter_cluster_distance = inter_cluster_distance
                            min_mnv = mnv
                            min_cluster_pair = (cluster1, cluster2)
            # print("min_mnv",min_mnv)
            if min_mnv >= self.MAX_MNV:
                break
            S.remove(min_cluster_pair[0])
            S.remove(min_cluster_pair[1])
            cluster_union = set(min_cluster_pair[0]).union(set(min_cluster_pair[1]))
            S.add(tuple(cluster_union))

        # Consider non-singleton collusion sets
        non_singleton_collusion_set = list(filter(lambda x: len(x)>1, S))
        sorted(non_singleton_collusion_set,key=lambda x: len(x),reverse=True)
        return non_singleton_collusion_set

In [11]:
# Function for finding candidate collusion set
def candidate_collusion_set(vertices, graph, hyper_parameters):
    frauds = []
    for k, m in tqdm(hyper_parameters):
        print("\n\nRunning on Hyper Parameters: k = ",k," \t m = ",m)
        collusions = MutualNearestNeighbour(k,vertices,graph,m).mutual_NN_avg()
        print("Collusion Sets:",collusions)
        collusionset = merge_tuples_to_set(collusions)
        frauds.append(collusionset)
    
    # return the intersection of vertices
    return set(reduce(lambda a,b: a.intersection(b), frauds))

In [12]:
# Testing the functions
hyper_parameters = [(2,1),(3,1),(4,1),(5,1),(6,1)]
vertices, graph = get_graph_from_dataset(PATH2)
print("\n\nCandidate Collusion Set: ",candidate_collusion_set(vertices,graph,hyper_parameters))

100%|██████████| 5/5 [00:00<00:00, 346.53it/s]



Running on Hyper Parameters: k =  2  	 m =  1
Collusion Sets: [(1037, 1029, 1054)]


Running on Hyper Parameters: k =  3  	 m =  1
Collusion Sets: [(1037, 1029, 1054), (1042, 1035)]


Running on Hyper Parameters: k =  4  	 m =  1
Collusion Sets: [(1074, 1037, 1029, 1054), (1042, 1035)]


Running on Hyper Parameters: k =  5  	 m =  1
Collusion Sets: [(1074, 1042, 1029, 1035, 1037, 1054)]


Running on Hyper Parameters: k =  6  	 m =  1
Collusion Sets: [(1074, 1042, 1029, 1035, 1037, 1054)]


Candidate Collusion Set:  {1029, 1037, 1054}
